In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import operator
# Non pythonic hack to reuse some utility code
if sys.path[0] != '../py_utils':
    sys.path.insert(0,'../py_utils')

import file_utils    
import utils
import itertools
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
from pathlib import Path
%matplotlib inline
pd.set_option('display.max_columns', 500)
print("Seaborn version: ", sns.__version__)

Seaborn version:  0.9.0


In [2]:
def get_team_record(summary_data_with_rankings, team_id, seed, is_opposition):
    team = summary_data_with_rankings[summary_data_with_rankings['team_id'] == team_id].copy()
    team['seed'] = seed
    if is_opposition:
        team = team.rename(columns={'season':'season_o','school':'school_o', 'team_id':'team_id_o', 'srs':'srs_o', 
                                    'sos':'sos_o','win_pct':'win_pct_o', 'pts_avg':'pts_avg_o', 
                                    'opp_pts_avg':'opp_pts_avg_o', 'fg_pct': 'fg_pct_o', 'ft_pct':'ft_pct_o', 
                                    'poss_avg':'poss_avg_o', 'off_rebs_avg':'off_rebs_avg_o', 'def_rebs_avg':'def_rebs_avg_o',
                                    'to_avg':'to_avg_o','steal_avg':'steal_avg_o','off_rating':'off_rating_o',
                                    'ft_att_avg':'ft_att_avg_o', 'SAG':'sag_o','WLK':'wlk_o','WOL':'wol_o',
                                    'RTH':'rth_o','COL':'col_o','POM':'pom_o','DOL':'dol_o','MOR':'mor_o','seed':'seed_o'})
    else:
        team = team.rename(columns={'season':'season_t','school':'school_t', 'team_id':'team_id_t', 'srs':'srs_t',
                                    'sos':'sos_t','win_pct':'win_pct_t', 'pts_avg':'pts_avg_t', 'opp_pts_avg':'opp_pts_avg_t',
                                    'fg_pct': 'fg_pct_t', 'ft_pct':'ft_pct_t', 'poss_avg':'poss_avg_t', 
                                    'off_rebs_avg':'off_rebs_avg_t', 'def_rebs_avg':'def_rebs_avg_t', 'to_avg':'to_avg_t',
                                    'steal_avg':'steal_avg_t','off_rating':'off_rating_t', 'ft_att_avg':'ft_att_avg_t', 
                                    'SAG':'sag_t','WLK':'wlk_t','WOL':'wol_t','RTH':'rth_t','COL':'col_t','POM':'pom_t', 
                                    'DOL':'dol_t','MOR':'mor_t','seed':'seed_t'}) 
        
    team = team.reset_index(drop=True)
    
    return team      

In [171]:
summary_team_file = '../Data/sr_summaries_kaggle_id_no_opp_2018.csv'
team_meta_data_file = '../Data/D1_teams.csv'
tournament_data_file = '../Data/tournament_results_2018.csv'
rankings_data_file = '../data/massey_seasons_with_id.csv'

feature_dictionary = utils.Feature_Dictionary()

In [172]:
#Train using 2003 through 2018 tournament data corresponding to seasons 2002-2003 through 2017-2018
start_tournament = 2003
stop_tournament = 2018

In [173]:
# The season column in the summary data corresponds to the start year of the season
summary_data = file_utils.read_summary_team_data(summary_team_file)

# Train the model using seasons up to and including 2017-2018 
summary_data = summary_data[summary_data['season'] < stop_tournament]
teams = file_utils.read_team_meta_data(team_meta_data_file)
summary_data.head()

,season,school,team_id,games,srs,sos,win_pct,pts_avg,opp_pts_avg,fg_pct,ft_pct,poss_avg,off_rebs_avg,def_rebs_avg,to_avg,steal_avg,off_rating,ft_att_avg
0,2000,Air Force,aa7af640-5762-4686-9181-39f7b8a8186e,29,-3.57,1.99,0.276,57,63,0.435,0.697,63.020690,6.0,16.0,13,7,90.446487,16
1,2000,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,28,-3.09,1.67,0.429,67,72,0.452,0.725,73.857143,9.0,20.0,15,5,90.715667,23
2,2000,Alabama,c2104cdc-c83d-40d2-a3cd-df986e29f5d3,36,14.02,6.27,0.694,76,68,0.433,0.698,86.200000,16.0,26.0,15,7,88.167053,24
3,2000,Alabama A&M,949c3398-85e4-4c63-ba71-9a82e06ddea4,28,-13.46,-11.79,0.607,73,74,0.391,0.626,94.264286,17.0,22.0,16,12,77.441843,23
4,2000,Alabama State,267d417a-8f85-4c87-a15a-068c089a74c6,31,-8.86,-13.61,0.710,68,63,0.441,0.658,80.690323,12.0,25.0,16,8,84.272807,22


#### Read in the NCAA Men's Tournament.

In [174]:
tourney_data = file_utils.read_tournament_results(tournament_data_file,start_tournament)
game_data = utils.compute_game_data(tourney_data, teams)

# The season year in the computer rankings file correspond to the tournament year (e.g. 2003 for 2002-2003 season)
computer_rankings = pd.read_csv(Path(rankings_data_file))
computer_rankings = computer_rankings[computer_rankings['season'] >= start_tournament]

# Recoding the tourney data to generate team and opp_team fields to replace win and lose fields
# Also add a start season field to the tourney data for merging with summary data
tourney_data = utils.recode_tourney_data(tourney_data)

tourney_data = file_utils.merge_tourney_summary_data(tourney_data, summary_data)
tourney_data = file_utils.join_tourney_team_data(tourney_data, teams)
tourney_comp_ratings = file_utils.merge_tourney_ranking_data(tourney_data, computer_rankings)
tourney_comp_ratings = utils.implement_top_conference_feature(game_data, tourney_comp_ratings)
tourney_comp_ratings = utils.implement_seed_threshold_feature(tourney_comp_ratings)
tourney_comp_ratings = utils.compute_delta_features(tourney_comp_ratings)

tourney_comp_ratings.dropna(inplace=True)
tourney_comp_ratings[tourney_comp_ratings.isnull().any(axis=1)]

# Get the top conference list to use when predicting current tournament results
top_tournament_conferences = utils.get_tournament_top_conference_list(game_data)


In [175]:
model_features = ['delta_margin_victory_avg', 'delta_fg_pct', 'delta_off_rebs_avg', 'delta_def_rebs_avg', 'delta_to_net_avg', 
                  'delta_win_pct', 'delta_off_rating', 'delta_ft_att_avg', 'delta_seed', 'delta_srs', 
                  'delta_sos', 'delta_sag', 'delta_wlk', 'delta_wol', 'delta_rth', 
                  'delta_col', 'delta_pom', 'delta_dol', 'delta_mor', 'top_conf_t', 'top_conf_o']

In [176]:
numeric_features = ['delta_margin_victory_avg', 'delta_fg_pct', 'delta_off_rebs_avg', 'delta_def_rebs_avg', 'delta_to_net_avg', 
                    'delta_win_pct', 'delta_off_rating', 'delta_ft_att_avg', 'delta_seed', 'delta_srs', 
                    'delta_sos', 'delta_sag', 'delta_wlk', 'delta_wol', 'delta_rth', 
                    'delta_col', 'delta_pom', 'delta_dol', 'delta_mor']

# Convert types to float to eliminate needless scaler warning 
for item in numeric_features:
    tourney_comp_ratings[item] = tourney_comp_ratings[item].astype(float)


In [177]:
feature_columns = numeric_features  + ['season_t', 'top_conf_t', 'top_conf_o']
feature_data = tourney_comp_ratings[feature_columns].copy()
feature_data.head()

,delta_margin_victory_avg,delta_fg_pct,delta_off_rebs_avg,delta_def_rebs_avg,delta_to_net_avg,delta_win_pct,delta_off_rating,delta_ft_att_avg,delta_seed,delta_srs,delta_sos,delta_sag,delta_wlk,delta_wol,delta_rth,delta_col,delta_pom,delta_dol,delta_mor,season_t,top_conf_t,top_conf_o
0,2.0,0.007,1.0,1.0,0.0,-0.061,2.364814,2.0,-1.0,3.05,0.86,-8.0,12.0,8.0,2.0,12.0,5.0,-4.0,-39.0,2003,0,0
1,4.0,0.017,-1.0,-1.0,-1.0,-0.009,7.042104,-5.0,-13.0,20.19,14.63,-131.0,-91.0,-104.0,-133.0,-73.0,-128.0,-85.0,-162.0,2003,1,0
2,5.0,-0.012,2.0,3.0,1.0,0.159,4.598761,6.0,-8.0,4.28,-1.38,-34.0,-38.0,-36.0,-33.0,-36.0,-33.0,-39.0,-24.0,2003,0,0
3,-6.0,0.022,1.0,-1.0,3.0,-0.120,-4.525801,0.0,3.0,-1.93,4.63,23.0,31.0,13.0,12.0,17.0,27.0,21.0,6.0,2003,1,0
4,0.0,-0.029,3.0,-2.0,1.0,-0.069,-0.666215,1.0,-2.0,-0.11,0.25,-7.0,-7.0,2.0,-6.0,-5.0,-8.0,2.0,-9.0,2003,0,0


In [178]:
X= feature_data[feature_data['season_t']<= stop_tournament]
y=tourney_comp_ratings[tourney_comp_ratings['season_t']<= stop_tournament]['game_result']
X.describe()


,delta_margin_victory_avg,delta_fg_pct,delta_off_rebs_avg,delta_def_rebs_avg,delta_to_net_avg,delta_win_pct,delta_off_rating,delta_ft_att_avg,delta_seed,delta_srs,delta_sos,delta_sag,delta_wlk,delta_wol,delta_rth,delta_col,delta_pom,delta_dol,delta_mor,season_t,top_conf_t,top_conf_o
count,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000,1045.000000
mean,-0.118660,-0.000617,-0.043062,-0.095694,-0.083254,-0.002332,-0.008594,-0.046890,0.241148,-0.237368,-0.113359,2.078469,1.850718,1.796172,2.167464,1.823923,1.715789,1.573206,1.692823,2010.607656,0.241148,0.238278
std,6.113963,0.031684,2.540300,2.553720,2.500911,0.137027,6.456617,3.629211,7.464683,11.096737,6.884305,69.523609,64.192081,61.678327,73.700633,61.313728,69.249290,64.041127,73.124929,4.607293,0.427985,0.426234
min,-21.000000,-0.094000,-8.000000,-9.000000,-7.000000,-0.493000,-21.932677,-10.000000,-15.000000,-36.970000,-19.570000,-309.000000,-282.000000,-283.000000,-310.000000,-257.000000,-318.000000,-293.000000,-302.000000,2003.000000,0.000000,0.000000
25%,-4.000000,-0.023000,-2.000000,-2.000000,-2.000000,-0.091000,-4.499795,-2.000000,-5.000000,-6.490000,-3.870000,-24.000000,-22.000000,-22.000000,-25.000000,-24.000000,-25.000000,-23.000000,-25.000000,2007.000000,0.000000,0.000000
50%,0.000000,-0.001000,0.000000,0.000000,0.000000,0.000000,0.148123,0.000000,0.000000,-0.220000,-0.220000,1.000000,1.000000,2.000000,2.000000,1.000000,-1.000000,1.000000,1.000000,2011.000000,0.000000,0.000000
75%,4.000000,0.021000,2.000000,2.000000,2.000000,0.086000,4.112294,2.000000,7.000000,5.930000,3.260000,28.000000,26.000000,26.000000,29.000000,26.000000,27.000000,25.000000,30.000000,2015.000000,0.000000,0.000000
max,22.000000,0.109000,9.000000,9.000000,8.000000,0.560000,19.316619,12.000000,15.000000,36.990000,21.640000,265.000000,248.000000,251.000000,289.000000,256.000000,249.000000,266.000000,267.000000,2018.000000,1.000000,1.000000


In [179]:
X= X.drop(columns=['season_t'])
scaler =StandardScaler(copy=True, with_mean=True, with_std=True)

scaler.fit(X[numeric_features])
X[numeric_features]= scaler.transform(X[numeric_features])


In [180]:
svm = SVC(kernel='linear', probability=True)
result = svm.fit(X,y)

#### Tournament Predictions

##### 2014 Tournament

In [354]:
class TournamentPool:
    
    def __init__(self, model_features, numeric_features, top_tournament_conferences, classifier, scaler):
        
        self.model_features = model_features
        self.numeric_features = numeric_features
        self.top_tournament_conferences = top_tournament_conferences
        self.classifier = classifier
        self.scaler = scaler
        
        self.team_dict = { 
            'season':'season_t','school':'school_t', 'kaggle_id':'team_id_t', 'srs':'srs_t',
            'sos':'sos_t','win_pct':'win_pct_t', 'pts_avg':'pts_avg_t', 'opp_pts_avg':'opp_pts_avg_t',
            'fg_pct': 'fg_pct_t', 'ft_pct':'ft_pct_t', 'poss_avg':'poss_avg_t', 
            'off_rebs_avg':'off_rebs_avg_t', 'def_rebs_avg':'def_rebs_avg_t', 'to_avg':'to_avg_t',
            'steal_avg':'steal_avg_t','off_rating':'off_rating_t', 'ft_att_avg':'ft_att_avg_t', 
            'SAG':'sag_t','WLK':'wlk_t','WOL':'wol_t','RTH':'rth_t','COL':'col_t','POM':'pom_t', 
            'DOL':'dol_t','MOR':'mor_t','Seed':'seed_t','TeamID':'kaggle_team_id_t'
        }
        
        self.opp_dict = {
            'season':'season_o','school':'school_o', 'kaggle_id':'team_id_o', 'srs':'srs_o', 
            'sos':'sos_o','win_pct':'win_pct_o', 'pts_avg':'pts_avg_o', 
            'opp_pts_avg':'opp_pts_avg_o', 'fg_pct': 'fg_pct_o', 'ft_pct':'ft_pct_o', 
            'poss_avg':'poss_avg_o', 'off_rebs_avg':'off_rebs_avg_o', 'def_rebs_avg':'def_rebs_avg_o',
            'to_avg':'to_avg_o','steal_avg':'steal_avg_o','off_rating':'off_rating_o',
            'ft_att_avg':'ft_att_avg_o', 'SAG':'sag_o','WLK':'wlk_o','WOL':'wol_o',
            'RTH':'rth_o','COL':'col_o','POM':'pom_o','DOL':'dol_o','MOR':'mor_o','Seed':'seed_o',
            'TeamID':'kaggle_team_id_o'
        }
        
        
        # Read tournament team file
        tournament_team_file = '../Data/tournament_teams.csv'
        self.tournament_teams = pd.read_csv(Path(tournament_team_file))
        
        # Read Computer Rankings File
        rankings_data_file = '../data/massey_seasons_with_id.csv'
        self.computer_rankings = pd.read_csv(Path(rankings_data_file))
        
        # Read team summary statistics
        summary_team_file = '../Data/sr_summaries_kaggle_id_no_opp_2018.csv'
        self.summary_data = file_utils.read_summary_team_data(summary_team_file)
        
        # Read team meta-data
        team_meta_data_file = '../Data/D1_teams.csv'
        self.teams = file_utils.read_team_meta_data(team_meta_data_file)
        
    def compute_top_conference_values(self, delta_game_record):
        team_conferences = self.teams.reset_index()
        t1_id = delta_game_record.iloc[0]['team_id_t']
        t2_id = delta_game_record.iloc[0]['team_id_o']
        t1_conference = team_conferences[team_conferences['id']== t1_id]['conf_name'].values[0]
        t2_conference = team_conferences[team_conferences['id']== t2_id]['conf_name'].values[0]
        # if both teams in same conference return 0,0 - no advantage
        if t1_conference == t2_conference:
            return 0,0
        t1_top_conference = t1_conference in self.top_tournament_conferences
        t2_top_conference = t2_conference in self.top_tournament_conferences
        if t1_top_conference == t2_top_conference:
            # both teams in or out of a top conference
            return 0,0 
        if t1_top_conference:
            return 1,0
        else:
            return 0,1
        
    
    def get_tournament_predictions(self, tournament_year):
        tourney_participants = self.tournament_teams[self.tournament_teams['Season'] == tournament_year]
        tourney_computer_rankings = self.computer_rankings[self.computer_rankings['season'] == tournament_year]
        
        tourney_teams_with_rankings = tourney_participants.merge(tourney_computer_rankings, left_on=['Season','id'], 
                                                                 right_on=['season','kaggle_id'], how='left',
                                                                 suffixes=('_x','_y'))
        
        
        tourney_summary_data = self.summary_data[summary_data['season']== (tournament_year -1)].copy()
        tourney_summary_data['tourney_season'] = tournament_year
        
        tourney_teams = tourney_teams_with_rankings.merge(tourney_summary_data, 
                                                          left_on=['Season','kaggle_id'], 
                                                          right_on=['tourney_season','team_id'], how='left',
                                                          suffixes=('','_y'))
        
        
        tourney_teams.drop(columns=['RPI','Season', 'season_y','Team','team_id','id','tourney_season', 'games'],
                              inplace=True)
        
        games = itertools.combinations(tourney_participants['id'],2)
        
        first_pass = True
        game_count = 1
        #for game in list(games)[0:10]:
        for game in games:
            
            print('game= ', game, '  game count:', game_count)
            game_count+= 1
            
            t1 = tourney_teams[tourney_teams['kaggle_id']== game[0]].reset_index()
            t2 = tourney_teams[tourney_teams['kaggle_id']== game[1]].reset_index()
            
            if t1.TeamID[0] < t2.TeamID[0]:
                team_t = t1.rename(columns=self.team_dict) 
                team_o = t2.rename(columns=self.opp_dict)
                
            else:
                team_t = t2.rename(columns=self.team_dict) 
                team_o = t1.rename(columns=self.opp_dict)
                
            game_record_df = pd.concat([team_t ,team_o], axis=1, sort=False)
            delta_game_record = utils.compute_delta_features(game_record_df)  
            t1_top,t2_top = self.compute_top_conference_values(delta_game_record)
            delta_game_record['top_conf_t'] = t1_top
            delta_game_record['top_conf_o'] = t2_top
        
            game_string = str(tournament_year) + '_' + str(team_t.kaggle_team_id_t[0]) + '_' + \
                str(team_o.kaggle_team_id_o[0])
            print('game_string= ', game_string)
                  
            delta_game_record['game_string'] = game_string
            if first_pass:
                prediction_records = delta_game_record.copy()
                first_pass = False
            else:
                prediction_records = prediction_records.append(delta_game_record.copy(), ignore_index=True)
              
        
        for item in self.numeric_features:
            prediction_records[item] = prediction_records[item].astype(float)
            
        prediction_records[self.numeric_features] = self.scaler.transform(prediction_records[self.numeric_features])
        
        predictions = self.classifier.predict(prediction_records[self.model_features])
        
        prediction_records['predictions'] = predictions
        print('predictions= ', predictions)
        
        prediction_probabilities = self.classifier.predict_proba(prediction_records[self.model_features])
        print('pred probs=', prediction_probabilities) 
        team_t_probs = [item[1] for item in prediction_probabilities]
        prediction_records['team_t_probs'] =  team_t_probs
        prediction_records['team_t_probs'] = prediction_records['team_t_probs'].round(4).apply(str)
        #prediction_records['game_string'] += prediction_records['team_t_probs']
        return prediction_records

In [355]:
tournament_pool = TournamentPool(model_features, numeric_features, top_tournament_conferences, svm, scaler)
prediction_records = tournament_pool.get_tournament_predictions(2014)


game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '0d8a328f-20ce-410b-bed1-15fc01308aaf')   game count: 1
game_string=  2014_1174_1462
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '0e4258fa-3154-4c16-b693-adecab184c6c')   game count: 2
game_string=  2014_1174_1235
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1551b3a0-6d10-4ffa-8dfa-73b56f004553')   game count: 3
game_string=  2014_1174_1272
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1da70895-f77f-44ef-b216-d63c02e696eb')   game count: 4
game_string=  2014_1174_1332
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1b1977ec-b32f-4ce3-95a4-d07bab4816cd')   game count: 5
game_string=  2014_1174_1307
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1e42cce3-236b-4f3e-9650-2c45e6332e1e')   game count: 6
game_string=  2014_1174_1387
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '2267a1f4-68f6-418b-aaf6-2aa0c4b291f1')   game count: 7
game_string=  2014_1174_1246
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '24051034-96bb-4f78-a

game_string=  2014_1174_1304
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 65
game_string=  2014_1174_1242
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 66
game_string=  2014_1174_1181
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 67
game_string=  2014_1174_1386
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '0e4258fa-3154-4c16-b693-adecab184c6c')   game count: 68
game_string=  2014_1235_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '1551b3a0-6d10-4ffa-8dfa-73b56f004553')   game count: 69
game_string=  2014_1272_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '1da70895-f77f-44ef-b216-d63c02e696eb')   game count: 70
game_string=  2014_1332_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '1b1977ec-b32f-4ce3-95a4-d07bab4816cd')   game count: 71
game_string=  2014_1307_1462
game=  ('0d8a328f-20ce-410b-bed1-

game_string=  2014_1417_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 127
game_string=  2014_1163_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 128
game_string=  2014_1455_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 129
game_string=  2014_1444_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 130
game_string=  2014_1304_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 131
game_string=  2014_1242_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 132
game_string=  2014_1181_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 133
game_string=  2014_1386_1462
game=  ('0e4258fa-3154-4c1

game_string=  2014_1235_1411
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 191
game_string=  2014_1235_1417
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 192
game_string=  2014_1163_1235
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 193
game_string=  2014_1235_1455
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 194
game_string=  2014_1235_1444
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 195
game_string=  2014_1235_1304
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 196
game_string=  2014_1235_1242
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 197
game_string=  2014_1181_1235
game=  ('0e4258fa-3154-4c1

game_string=  2014_1272_1411
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 255
game_string=  2014_1272_1417
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 256
game_string=  2014_1163_1272
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 257
game_string=  2014_1272_1455
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 258
game_string=  2014_1272_1444
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 259
game_string=  2014_1272_1304
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 260
game_string=  2014_1242_1272
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 261
game_string=  2014_1181_1272
game=  ('1551b3a0-6d10-4ff

...

game_string=  2014_1277_1455
game=  ('a41d5a05-4c11-4171-a57e-e7a1ea325a6d', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 1974
game_string=  2014_1277_1444
game=  ('a41d5a05-4c11-4171-a57e-e7a1ea325a6d', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 1975
game_string=  2014_1277_1304
game=  ('a41d5a05-4c11-4171-a57e-e7a1ea325a6d', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 1976
game_string=  2014_1242_1277
game=  ('a41d5a05-4c11-4171-a57e-e7a1ea325a6d', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 1977
game_string=  2014_1181_1277
game=  ('a41d5a05-4c11-4171-a57e-e7a1ea325a6d', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 1978
game_string=  2014_1277_1386
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'ad4bc983-8d2e-4e6f-a8f9-80840a786c64')   game count: 1979
game_string=  2014_1113_1328
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'b795ddbc-baab-4499-8803-52e8608520ab')   game count: 1980
game_string=  2014_1257_1328
game=  ('b2fda957-e

game_string=  2014_1257_1361
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'e9ca48b2-00ba-41c0-a02b-6885a2da1ff1')   game count: 2038
game_string=  2014_1257_1314
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'eb157f98-0697-459c-9293-ddb162ceb28b')   game count: 2039
game_string=  2014_1257_1411
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 2040
game_string=  2014_1257_1417
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 2041
game_string=  2014_1163_1257
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2042
game_string=  2014_1257_1455
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 2043
game_string=  2014_1257_1444
game=  ('b795ddbc-baab-4499-8803-52e8608520ab', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 2044
game_string=  2014_1257_1304
game=  ('b795ddbc-b

game_string=  2014_1411_1433
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 2100
game_string=  2014_1417_1433
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 2101
game_string=  2014_1163_1433
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2102
game_string=  2014_1433_1455
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 2103
game_string=  2014_1433_1444
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 2104
game_string=  2014_1304_1433
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 2105
game_string=  2014_1242_1433
game=  ('c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 2106
game_string=  2014_1181_1433
game=  ('c1c1e6df-a

game_string=  2014_1184_1243
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'e4c315a0-e83f-4d14-856e-34a0463059e3')   game count: 2162
game_string=  2014_1243_1300
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'e6914f16-ca37-4d2e-ba26-a425910283e2')   game count: 2163
game_string=  2014_1243_1361
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'e9ca48b2-00ba-41c0-a02b-6885a2da1ff1')   game count: 2164
game_string=  2014_1243_1314
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'eb157f98-0697-459c-9293-ddb162ceb28b')   game count: 2165
game_string=  2014_1243_1411
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 2166
game_string=  2014_1243_1417
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 2167
game_string=  2014_1163_1243
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2168
game_string=  2014_1243_1455
game=  ('d203f38a-a

game_string=  2014_1361_1411
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 2226
game_string=  2014_1361_1417
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 2227
game_string=  2014_1163_1361
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2228
game_string=  2014_1361_1455
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 2229
game_string=  2014_1361_1444
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 2230
game_string=  2014_1304_1361
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 2231
game_string=  2014_1242_1361
game=  ('e6914f16-ca37-4d2e-ba26-a425910283e2', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 2232
game_string=  2014_1181_1361
game=  ('e6914f16-c

In [356]:
prediction_records.shape

(2278, 86)

In [357]:
prediction_records.head()

,index,kaggle_team_id_t,seed_t,sag_t,wlk_t,wol_t,rth_t,col_t,pom_t,dol_t,mor_t,season_t,win_pct_t,team_id_t,school_t,srs_t,sos_t,win_pct_y,pts_avg_t,opp_pts_avg_t,fg_pct_t,ft_pct_t,poss_avg_t,off_rebs_avg_t,def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,index,kaggle_team_id_o,seed_o,sag_o,wlk_o,wol_o,rth_o,col_o,pom_o,dol_o,mor_o,win_pct_o,team_id_o,school_o,srs_o,sos_o,win_pct_y,pts_avg_o,opp_pts_avg_o,fg_pct_o,ft_pct_o,poss_avg_o,off_rebs_avg_o,def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,margin_victory_avg_t,margin_victory_avg_o,delta_margin_victory_avg,delta_fg_pct,delta_off_rebs_avg,delta_def_rebs_avg,delta_ft_pct,to_net_avg_t,to_net_avg_o,delta_to_net_avg,delta_win_pct,delta_off_rating,delta_ft_att_avg,delta_seed,delta_srs,delta_sos,delta_sag,delta_wlk,delta_wol,delta_rth,delta_col,delta_pom,delta_dol,delta_mor,top_conf_t,top_conf_o,game_string,predictions,team_t_probs
0,0,1174,13,106,98.0,72,111,80.0,103.0,75.0,119,2014,0.735294,0affc15f-641d-4211-970d-fb9fb8d36842,Delaware,2.73,-1.1,0.714,79,75,0.444,0.711,80.342857,11.0,25.0,9,5,98.328592,26,1,1462,12,48,38.0,48,39,44.0,56.0,42.0,50,0.636364,0d8a328f-20ce-410b-bed1-15fc01308aaf,Xavier,11.84,8.14,0.618,71,68,0.470,0.688,75.852941,11.0,24.0,12,5,93.602171,22,4,3,0.183056,-0.801496,0.016960,0.429263,0.023,4,7,-1.166832,0.739349,0.733710,1.115622,0.101708,-0.799954,-1.326352,0.804738,0.906298,0.360166,0.947970,0.557664,0.654243,0.490963,0.920882,0,1,2014_1174_1462,-1,0.1682
1,0,1174,13,106,98.0,72,111,80.0,103.0,75.0,119,2014,0.735294,0affc15f-641d-4211-970d-fb9fb8d36842,Delaware,2.73,-1.1,0.714,79,75,0.444,0.711,80.342857,11.0,25.0,9,5,98.328592,26,2,1235,3,13,14.0,9,10,6.0,23.0,5.0,20,0.787879,0e4258fa-3154-4c16-b693-adecab184c6c,Iowa State,18.52,9.88,0.778,83,74,0.474,0.693,81.794444,10.0,28.0,10,5,101.473884,22,4,9,-0.798774,-0.927801,0.410802,-1.137829,0.018,4,5,-0.366740,-0.366910,-0.486044,1.115622,1.307962,-1.402221,-1.579222,1.308405,1.280354,0.992782,1.341642,1.177724,1.131011,1.068994,1.331335,0,1,2014_1174_1235,-1,0.0347
2,0,1174,13,106,98.0,72,111,80.0,103.0,75.0,119,2014,0.735294,0affc15f-641d-4211-970d-fb9fb8d36842,Delaware,2.73,-1.1,0.714,79,75,0.444,0.711,80.342857,11.0,25.0,9,5,98.328592,26,3,1272,8,41,43.0,37,37,35.0,40.0,28.0,39,0.718750,1551b3a0-6d10-4ffa-8dfa-73b56f004553,Memphis,13.02,7.14,0.706,76,70,0.477,0.644,81.717647,12.0,24.0,13,8,93.003167,23,4,6,-0.307859,-1.022530,-0.376883,0.429263,0.067,4,5,-0.366740,0.137821,0.826528,0.839948,0.637821,-0.906343,-1.181024,0.905472,0.828369,0.538596,0.975120,0.704520,0.885403,0.709677,1.071381,0,1,2014_1174_1272,-1,0.1543
3,0,1174,13,106,98.0,72,111,80.0,103.0,75.0,119,2014,0.735294,0affc15f-641d-4211-970d-fb9fb8d36842,Delaware,2.73,-1.1,0.714,79,75,0.444,0.711,80.342857,11.0,25.0,9,5,98.328592,26,4,1332,7,30,23.0,32,22,27.0,30.0,29.0,32,0.718750,1da70895-f77f-44ef-b216-d63c02e696eb,Oregon,15.75,8.05,0.706,81,74,0.469,0.772,79.764706,10.0,23.0,11,7,101.548673,25,4,7,-0.471498,-0.769919,0.410802,0.821036,-0.061,4,4,0.033305,0.137821,-0.497633,0.288600,0.771849,-1.152479,-1.313272,1.063767,1.140083,0.619701,1.178743,0.835059,1.029878,0.694055,1.167154,0,1,2014_1174_1332,-1,0.0994
4,0,1174,13,106,98.0,72,111,80.0,103.0,75.0,119,2014,0.735294,0affc15f-641d-4211-970d-fb9fb8d36842,Delaware,2.73,-1.1,0.714,79,75,0.444,0.711,80.342857,11.0,25.0,9,5,98.328592,26,5,1307,7,31,31.0,19,27,17.0,28.0,20.0,28,0.818182,1b1977ec-b32f-4ce3-95a4-d07bab4816cd,New Mexico,12.72,4.84,0.794,74,66,0.458,0.724,75.211765,10.0,27.0,10,4,98.388863,25,4,8,-0.635136,-0.422579,0.410802,-0.746056,-0.013,4,6,-0.766786,-0.588161,-0.008008,0.288600,0.771849,-0.879295,-0.846771,1.049377,1.015397,0.830573,1.110869,0.998233,1.058773,0.834657,1.221881,0,0,2014_1174_1307,-1,0.1351


In [313]:
prediction_records[(prediction_records['kaggle_team_id_t']==1174) & (prediction_records['kaggle_team_id_o']==1277)]

,index,kaggle_team_id_t,seed_t,sag_t,wlk_t,wol_t,rth_t,col_t,pom_t,dol_t,mor_t,season_t,win_pct_t,team_id_t,school_t,srs_t,sos_t,win_pct_y,pts_avg_t,opp_pts_avg_t,fg_pct_t,ft_pct_t,poss_avg_t,off_rebs_avg_t,def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,index,kaggle_team_id_o,seed_o,sag_o,wlk_o,wol_o,rth_o,col_o,pom_o,dol_o,mor_o,win_pct_o,team_id_o,school_o,srs_o,sos_o,win_pct_y,pts_avg_o,opp_pts_avg_o,fg_pct_o,ft_pct_o,poss_avg_o,off_rebs_avg_o,def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,margin_victory_avg_t,margin_victory_avg_o,delta_margin_victory_avg,delta_fg_pct,delta_off_rebs_avg,delta_def_rebs_avg,delta_ft_pct,to_net_avg_t,to_net_avg_o,delta_to_net_avg,delta_win_pct,delta_off_rating,delta_ft_att_avg,delta_seed,delta_srs,delta_sos,delta_sag,delta_wlk,delta_wol,delta_rth,delta_col,delta_pom,delta_dol,delta_mor,top_conf_t,top_conf_o,game_string,predictions,team_t_probs
41,0,1174,13,106,98.0,72,111,80.0,103.0,75.0,119,2014,0.735294,0affc15f-641d-4211-970d-fb9fb8d36842,Delaware,2.73,-1.1,0.714,79,75,0.444,0.711,80.342857,11.0,25.0,9,5,98.328592,26,42,1277,4,8,10.0,16,12,12.0,10.0,11.0,5,0.764706,a41d5a05-4c11-4171-a57e-e7a1ea325a6d,Michigan State,19.35,9.38,0.763,75,65,0.474,0.707,75.794737,10.0,26.0,11,6,98.951462,19,4,10,-0.962413,-0.927801,0.410802,-0.354283,0.004,4,5,-0.36674,-0.197717,-0.095184,1.942644,1.173934,-1.477054,-1.506558,1.380358,1.342697,0.879236,1.314492,1.07982,1.318828,0.975259,1.536562,0,1,"2014_1174_1277,",-1,0.040516


In [330]:
 prediction_records[prediction_records['school_t']=='Milwaukee ']

,index,kaggle_team_id_t,seed_t,sag_t,wlk_t,wol_t,rth_t,col_t,pom_t,dol_t,mor_t,season_t,win_pct_t,team_id_t,school_t,srs_t,sos_t,win_pct_y,pts_avg_t,opp_pts_avg_t,fg_pct_t,ft_pct_t,poss_avg_t,off_rebs_avg_t,def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,index,kaggle_team_id_o,seed_o,sag_o,wlk_o,wol_o,rth_o,col_o,pom_o,dol_o,mor_o,win_pct_o,team_id_o,school_o,srs_o,sos_o,win_pct_y,pts_avg_o,opp_pts_avg_o,fg_pct_o,ft_pct_o,poss_avg_o,off_rebs_avg_o,def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,margin_victory_avg_t,margin_victory_avg_o,delta_margin_victory_avg,delta_fg_pct,delta_off_rebs_avg,delta_def_rebs_avg,delta_ft_pct,to_net_avg_t,to_net_avg_o,delta_to_net_avg,delta_win_pct,delta_off_rating,delta_ft_att_avg,delta_seed,delta_srs,delta_sos,delta_sag,delta_wlk,delta_wol,delta_rth,delta_col,delta_pom,delta_dol,delta_mor,top_conf_t,top_conf_o,game_string,predictions,team_t_probs
86,21,1454,15,165,154.0,127,145,130.0,168.0,127.0,172,2014,0.617647,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,Milwaukee,-1.53,-0.44,0.6,70,70,0.428,0.734,76.748571,9.0,23.0,13,5,91.206909,23,1,1462,12,48,38.0,48,39,44.0,56.0,42.0,50,0.636364,0d8a328f-20ce-410b-bed1-15fc01308aaf,Xavier,11.84,8.14,0.618,71,68,0.470,0.688,75.852941,11.0,24.0,12,5,93.602171,22,0,3,-0.471498,-1.306717,-0.770726,-0.354283,0.046,8,7,0.433351,-0.119628,-0.369824,0.288600,0.369764,-1.184034,-1.230436,1.653777,1.779097,1.252317,1.409516,1.373532,1.593331,1.303330,1.646016,0,1,"2014_1454_1462,",-1,0.143714
1212,21,1454,15,165,154.0,127,145,130.0,168.0,127.0,172,2014,0.617647,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,Milwaukee,-1.53,-0.44,0.6,70,70,0.428,0.734,76.748571,9.0,23.0,13,5,91.206909,23,37,1459,15,192,188.0,180,214,181.0,181.0,193.0,165,0.625000,9916713f-b243-452b-93ae-a2c3ccabf68b,Wofford,-4.32,-5.95,0.606,66,62,0.448,0.664,72.478788,10.0,22.0,10,6,91.061126,17,0,4,-0.635136,-0.612037,-0.376883,0.429263,0.070,8,4,1.633488,-0.036659,0.023922,1.666970,-0.032321,0.272947,0.817229,-0.418453,-0.558759,-0.888844,-0.966091,-0.861948,-0.212606,-1.055659,0.072612,0,0,"2014_1454_1459,",-1,0.204246
1226,21,1454,15,165,154.0,127,145,130.0,168.0,127.0,172,2014,0.617647,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,Milwaukee,-1.53,-0.44,0.6,70,70,0.428,0.734,76.748571,9.0,23.0,13,5,91.206909,23,51,1458,2,9,9.0,13,13,7.0,13.0,7.0,9,0.787879,c7569eae-5b93-4197-b204-6f3a62146b25,Wisconsin,19.69,10.19,0.789,73,64,0.459,0.746,70.273684,8.0,24.0,8,4,103.879569,22,0,9,-1.453328,-0.959377,0.410802,-0.354283,-0.012,8,4,1.633488,-1.225887,-1.962348,0.288600,1.710047,-1.891788,-1.528357,2.215006,2.231082,1.820049,1.762464,1.977275,2.214573,1.850116,2.206968,0,1,"2014_1454_1458,",-1,0.016035
1237,21,1454,15,165,154.0,127,145,130.0,168.0,127.0,172,2014,0.617647,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,Milwaukee,-1.53,-0.44,0.6,70,70,0.428,0.734,76.748571,9.0,23.0,13,5,91.206909,23,62,1455,1,12,11.0,1,1,3.0,5.0,2.0,12,1.000000,f38b58de-3273-4723-a78b-2f918e9aa2d5,Wichita State,16.65,1.74,0.972,75,59,0.461,0.725,74.511111,11.0,26.0,10,6,100.656129,25,0,16,-2.598796,-1.022530,-0.770726,-1.137829,0.009,8,4,1.633488,-2.774649,-1.462863,-0.538422,1.844076,-1.617703,-0.300340,2.171835,2.199911,2.014700,1.925363,2.042544,2.330153,1.928228,2.165923,0,1,"2014_1454_1455,",-1,0.038652


In [324]:
name = prediction_records[prediction_records['kaggle_team_id_t']==1181]['school_t'][131]
name

'Duke '

In [331]:
prediction_records[(prediction_records['kaggle_team_id_t']==1437) & (prediction_records['kaggle_team_id_o']==1454)]

,index,kaggle_team_id_t,seed_t,sag_t,wlk_t,wol_t,rth_t,col_t,pom_t,dol_t,mor_t,season_t,win_pct_t,team_id_t,school_t,srs_t,sos_t,win_pct_y,pts_avg_t,opp_pts_avg_t,fg_pct_t,ft_pct_t,poss_avg_t,off_rebs_avg_t,def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,index,kaggle_team_id_o,seed_o,sag_o,wlk_o,wol_o,rth_o,col_o,pom_o,dol_o,mor_o,win_pct_o,team_id_o,school_o,srs_o,sos_o,win_pct_y,pts_avg_o,opp_pts_avg_o,fg_pct_o,ft_pct_o,poss_avg_o,off_rebs_avg_o,def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,margin_victory_avg_t,margin_victory_avg_o,delta_margin_victory_avg,delta_fg_pct,delta_off_rebs_avg,delta_def_rebs_avg,delta_ft_pct,to_net_avg_t,to_net_avg_o,delta_to_net_avg,delta_win_pct,delta_off_rating,delta_ft_att_avg,delta_seed,delta_srs,delta_sos,delta_sag,delta_wlk,delta_wol,delta_rth,delta_col,delta_pom,delta_dol,delta_mor,top_conf_t,top_conf_o,game_string,predictions,team_t_probs
800,13,1437,2,4,5.0,4,4,4.0,7.0,4.0,6,2014,0.875,4383eb6a-7fd8-4ff2-94c5-43c933121e88,Villanova,18.97,7.5,0.853,77,66,0.456,0.716,78.005882,11.0,26.0,11,6,98.710504,24,21,1454,15,165,154.0,127,145,130.0,168.0,127.0,172,0.617647,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,Milwaukee,-1.53,-0.44,0.6,70,70,0.428,0.734,76.748571,9.0,23.0,13,5,91.206909,23,11,0,1.819439,0.903628,0.804645,1.21281,-0.018,5,8,-1.166832,1.89604,1.164044,0.2886,-1.774689,1.869675,1.170374,-2.346779,-2.351115,-2.024308,-1.943484,-2.08575,-2.350836,-1.946138,-2.294335,1,0,"2014_1437_1454,",1,0.989273


In [358]:
kaggle_submit_df = prediction_records[['game_string','team_t_probs']].copy()
kaggle_submit_df.rename(columns={'game_string':'ID','team_t_probs':'pred' }, inplace=True)
kaggle_submit_df.to_csv('../submissions/tourney_2014.csv', index=False)

In [362]:
for year in range(2014,2019):
    prediction_records = tournament_pool.get_tournament_predictions(year)
    kaggle_submit_df = prediction_records[['game_string','team_t_probs']].copy()
    kaggle_submit_df.rename(columns={'game_string':'ID','team_t_probs':'pred' }, inplace=True)
    file_name = '../submissions/tourney_'+str(year)+'.csv'
    kaggle_submit_df.to_csv(file_name, index=False)
   

game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '0d8a328f-20ce-410b-bed1-15fc01308aaf')   game count: 1
game_string=  2014_1174_1462
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '0e4258fa-3154-4c16-b693-adecab184c6c')   game count: 2
game_string=  2014_1174_1235
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1551b3a0-6d10-4ffa-8dfa-73b56f004553')   game count: 3
game_string=  2014_1174_1272
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1da70895-f77f-44ef-b216-d63c02e696eb')   game count: 4
game_string=  2014_1174_1332
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1b1977ec-b32f-4ce3-95a4-d07bab4816cd')   game count: 5
game_string=  2014_1174_1307
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '1e42cce3-236b-4f3e-9650-2c45e6332e1e')   game count: 6
game_string=  2014_1174_1387
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '2267a1f4-68f6-418b-aaf6-2aa0c4b291f1')   game count: 7
game_string=  2014_1174_1246
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', '24051034-96bb-4f78-a

game_string=  2014_1174_1304
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 65
game_string=  2014_1174_1242
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 66
game_string=  2014_1174_1181
game=  ('0affc15f-641d-4211-970d-fb9fb8d36842', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 67
game_string=  2014_1174_1386
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '0e4258fa-3154-4c16-b693-adecab184c6c')   game count: 68
game_string=  2014_1235_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '1551b3a0-6d10-4ffa-8dfa-73b56f004553')   game count: 69
game_string=  2014_1272_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '1da70895-f77f-44ef-b216-d63c02e696eb')   game count: 70
game_string=  2014_1332_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', '1b1977ec-b32f-4ce3-95a4-d07bab4816cd')   game count: 71
game_string=  2014_1307_1462
game=  ('0d8a328f-20ce-410b-bed1-

game_string=  2014_1455_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 129
game_string=  2014_1444_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 130
game_string=  2014_1304_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 131
game_string=  2014_1242_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 132
game_string=  2014_1181_1462
game=  ('0d8a328f-20ce-410b-bed1-15fc01308aaf', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 133
game_string=  2014_1386_1462
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', '1551b3a0-6d10-4ffa-8dfa-73b56f004553')   game count: 134
game_string=  2014_1235_1272
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', '1da70895-f77f-44ef-b216-d63c02e696eb')   game count: 135
game_string=  2014_1235_1332
game=  ('0e4258fa-3154-4c1

game_string=  2014_1163_1235
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 193
game_string=  2014_1235_1455
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 194
game_string=  2014_1235_1444
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 195
game_string=  2014_1235_1304
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 196
game_string=  2014_1235_1242
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 197
game_string=  2014_1181_1235
game=  ('0e4258fa-3154-4c16-b693-adecab184c6c', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 198
game_string=  2014_1235_1386
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', '1da70895-f77f-44ef-b216-d63c02e696eb')   game count: 199
game_string=  2014_1272_1332
game=  ('1551b3a0-6d10-4ff

game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'efba5350-79bc-47aa-a189-db177b95b491')   game count: 256
game_string=  2014_1163_1272
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 257
game_string=  2014_1272_1455
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f2164f01-2cbc-4967-bb1c-93656d790921')   game count: 258
game_string=  2014_1272_1444
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f8b1bf00-5b73-4ac4-98c0-ec554027ae32')   game count: 259
game_string=  2014_1272_1304
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 260
game_string=  2014_1242_1272
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 261
game_string=  2014_1181_1272
game=  ('1551b3a0-6d10-4ffa-8dfa-73b56f004553', 'f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8')   game count: 262
game_string=  2014_1272_1386
game=  ('1da70895-f77f-44ef-b216-d63c02e696eb', '1b1977

...

game_string=  2018_1314_1328
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 1994
game_string=  2018_1328_1417
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'ef184799-d79b-49d9-a662-991eaf4044cd')   game count: 1995
game_string=  2018_1328_1401
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'f770d3a2-221c-49ce-9c7a-680f7417fe9e')   game count: 1996
game_string=  2018_1328_1452
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'f7fcbb6e-4a55-41ea-ba21-f763ed914467')   game count: 1997
game_string=  2018_1328_1395
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 1998
game_string=  2018_1328_1455
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 1999
game_string=  2018_1242_1328
game=  ('b2fda957-e15c-4fb2-8a13-6e58496f561e', 'f8c705b7-87d3-411a-9c8b-5124daab0469')   game count: 2000
game_string=  2018_1328_1403
game=  ('b2fda957-e

game_string=  2018_1276_1335
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'eb157f98-0697-459c-9293-ddb162ceb28b')   game count: 2058
game_string=  2018_1276_1411
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'e9ca48b2-00ba-41c0-a02b-6885a2da1ff1')   game count: 2059
game_string=  2018_1276_1314
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'ec0d6b67-4b16-4b50-92b2-1a651dae6b0f')   game count: 2060
game_string=  2018_1276_1417
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'ef184799-d79b-49d9-a662-991eaf4044cd')   game count: 2061
game_string=  2018_1276_1401
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'f770d3a2-221c-49ce-9c7a-680f7417fe9e')   game count: 2062
game_string=  2018_1276_1452
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'f7fcbb6e-4a55-41ea-ba21-f763ed914467')   game count: 2063
game_string=  2018_1276_1395
game=  ('bdc2561d-f603-4fab-a262-f1d2af462277', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2064
game_string=  2018_1276_1455
game=  ('bdc2561d-f

game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'f7fcbb6e-4a55-41ea-ba21-f763ed914467')   game count: 2120
game_string=  2018_1243_1395
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2121
game_string=  2018_1243_1455
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 2122
game_string=  2018_1242_1243
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'f8c705b7-87d3-411a-9c8b-5124daab0469')   game count: 2123
game_string=  2018_1243_1403
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'f2cf9ba2-ac8c-4d92-8e7a-48dd88e8f40b')   game count: 2124
game_string=  2018_1243_1347
game=  ('d203f38a-a166-4258-bca2-e161b591ecfb', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 2125
game_string=  2018_1181_1243
game=  ('c70c23c4-ab09-4797-a1af-792c848ab17f', 'dcf5c2e7-c227-4c20-af26-715d5f859412')   game count: 2126
game_string=  2018_1116_1155
game=  ('c70c23c4-ab09-4797-a1af-792c848ab17f', 

game_string=  2018_1300_1401
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'f770d3a2-221c-49ce-9c7a-680f7417fe9e')   game count: 2181
game_string=  2018_1300_1452
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'f7fcbb6e-4a55-41ea-ba21-f763ed914467')   game count: 2182
game_string=  2018_1300_1395
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2183
game_string=  2018_1300_1455
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 2184
game_string=  2018_1242_1300
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'f8c705b7-87d3-411a-9c8b-5124daab0469')   game count: 2185
game_string=  2018_1300_1403
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'f2cf9ba2-ac8c-4d92-8e7a-48dd88e8f40b')   game count: 2186
game_string=  2018_1300_1347
game=  ('e4c315a0-e83f-4d14-856e-34a0463059e3', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 2187
game_string=  2018_1181_1300
game=  ('e3848467-6

game=  ('e9ca48b2-00ba-41c0-a02b-6885a2da1ff1', 'faeb1160-5d15-4f26-99fc-c441cf21fc7f')   game count: 2242
game_string=  2018_1181_1314
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 'ef184799-d79b-49d9-a662-991eaf4044cd')   game count: 2243
game_string=  2018_1401_1417
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 'f770d3a2-221c-49ce-9c7a-680f7417fe9e')   game count: 2244
game_string=  2018_1417_1452
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 'f7fcbb6e-4a55-41ea-ba21-f763ed914467')   game count: 2245
game_string=  2018_1395_1417
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 'f38b58de-3273-4723-a78b-2f918e9aa2d5')   game count: 2246
game_string=  2018_1417_1455
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 'fae4855b-1b64-4b40-a632-9ed345e1e952')   game count: 2247
game_string=  2018_1242_1417
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 'f8c705b7-87d3-411a-9c8b-5124daab0469')   game count: 2248
game_string=  2018_1403_1417
game=  ('ec0d6b67-4b16-4b50-92b2-1a651dae6b0f', 